### Algorithm: Transformers including normalization v1
### Dataset : Cricket
### Author: Edgar Acuna 
### Created: April 16, 2025

In [420]:
import tensorflow

In [421]:

import pandas as pd

In [422]:
import pandas as pd
from aeon.datasets import load_classification
from sklearn import preprocessing
from matplotlib import pyplot

def load_data(dataset, split="Train"):
    if split=="Train":
        X, y = load_classification(dataset, split="Train")
    else:
        X, y = load_classification(dataset, split="Test")
    print(" Shape of X = ", X.shape)
    print(" Shape of y = ", y.shape)
    #Swap dimensions so features are the third dimension
    X = X.swapaxes(1, 2)
    print(" New shape of X = ", X.shape)
    #prepare y
    y = pd.DataFrame(y)
    enc = preprocessing.OneHotEncoder(dtype=int)
    enc.fit(y)
    y = enc.transform(y).toarray()
    print(" New shape of y = ", y.shape)
    return X, y


In [423]:
# loading Training data
X, y = load_data("HandMovementDirection",split="Train")

 Shape of X =  (160, 10, 400)
 Shape of y =  (160,)
 New shape of X =  (160, 400, 10)
 New shape of y =  (160, 4)


In [424]:
import numpy as np
yo=np.argmax(y,axis=1)
yo=np.array(yo,dtype=int)
print(yo)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


In [425]:
# loading Testing data
Xt, yt = load_data("HandMovementDirection",split="Test")

 Shape of X =  (74, 10, 400)
 Shape of y =  (74,)
 New shape of X =  (74, 400, 10)
 New shape of y =  (74, 4)


In [426]:
yto=np.argmax(yt,axis=1)
yto=np.array(yto,dtype=int)
#print(yto)

In [427]:
import numpy as np
m=X.mean(axis=0)
s=X.std(axis=0)
X=(X-m)/s
Xt=(Xt-m)/s

In [428]:
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, roc_auc_score, ConfusionMatrixDisplay
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.layers import Flatten
from keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
# Configuration options
#trainX, trainy, testX, testy = trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.3, random_state=42)
trainX=X
trainy=y
testX=Xt
testy=yt
feature_vector_length = trainX.shape[1]
num_classes = trainy.shape[1]
# Convert target classes to categorical ones
print(trainy.shape)
#trainy= to_categorical(trainy, num_classes)
#testy = to_categorical(testy, num_classes)
print('Train dimension:')
print(trainX.shape)
print('Test dimension:')
print(testX.shape)
#X_train=trainX.to_numpy()
#X_test=testX.to_numpy()
X_train = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2]))
X_test = testX.reshape((testX.shape[0], testX.shape[1], trainX.shape[2]))

(160, 4)
Train dimension:
(160, 400, 10)
Test dimension:
(74, 400, 10)


In [429]:
X_test.shape, testy.shape

((74, 400, 10), (74, 4))

In [430]:
X_train.shape, trainy.shape

((160, 400, 10), (160, 4))

In [431]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [432]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    n_length=100
    n_features=6
    inputs = keras.Input(shape=input_shape)
    x= Conv1D(filters=16, kernel_size=1, activation='relu')(inputs)
    x=MaxPooling1D(pool_size=2)(x)
    #x= Dropout(0.5)(x)
    x= Conv1D(filters=64, kernel_size=1, activation='relu')(x)
    x=MaxPooling1D(pool_size=2)(x)
    #x= Dropout(0.5)(x)
    x= Conv1D(filters=256, kernel_size=1, activation='relu')(x)
    x= MaxPooling1D(pool_size=2)(x)
    #x= Dropout(0.5)(x)
    #x = layers.LayerNormalization(epsilon=1e-6)(x)
    #x=Flatten()(x)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [433]:
input_shape = X_train.shape[1:]
print('input', input_shape)
import time
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0,
    dropout=0,
)
start_time = time.time()
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

#callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    trainy,
    validation_split=0,
    epochs=200,
    batch_size=8,
    #callbacks=callbacks,
)


input (400, 10)


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 400, 10)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_121 (Conv1D) │ (None, 400, 16)   │        176 │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_33    │ (None, 200, 16)   │          0 │ conv1d_121[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_122 (Conv1D) │ (None, 200, 64)   │      1,088 │ max_pooling1d_33… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_34    │ (None, 100, 64)   │          0 │ conv1d_122[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_123 (Conv1D) │ (None, 100, 256)  │     16,640 │ max_pooling1d_34… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_35    │ (None, 50, 256)   │          0 │ conv1d_123[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 256)   │        512 │ max_pooling1d_35… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 256)   │  1,051,904 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_144         │ (None, 50, 256)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_88 (Add)        │ (None, 50, 256)   │          0 │ dropout_144[0][0… │
│                     │                   │            │ max_pooling1d_35… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 256)   │        512 │ add_88[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_124 (Conv1D) │ (None, 50, 4)     │      1,028 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_145         │ (None, 50, 4)     │          0 │ conv1d_124[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_125 (Conv1D) │ (None, 50, 256)   │      1,280 │ dropout_145[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_89 (Add)        │ (None, 50, 256)   │          0 │ conv1d_125[0][0], │
│                     │                   │            │ add_88[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 256)   │        512 │ add_89[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 256)   │  1,051,904 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 4,245,892 (16.20 MB)

 Trainable params: 4,245,892 (16.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.2689 - loss: 1.3874
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.2479 - loss: 1.4083
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.2879 - loss: 1.3853
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.2957 - loss: 1.3817
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.2953 - loss: 1.3891
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.2530 - loss: 1.3863
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.2827 - loss: 1.3864
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.2487 - loss: 1.3897
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.2788 - loss: 1.3785
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.2653 - loss: 1.3805
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.3832 - loss: 1.3722
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/ste

In [434]:
results=model.evaluate(X_test, testy, verbose=1)
print("Training time --- %s seconds ---" % (time.time() - start_time))
print(results)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.3678 - loss: 6.7734
Training time --- 552.1712744235992 seconds ---
[6.666325092315674, 0.3918918967247009]


In [435]:
#Computing F1-score
import numpy as np
import sklearn
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, roc_auc_score
train_features = np.array(X_train)
test_features = np.array(X_test)
train_labels=np.array(trainy)
#train_labels=pd.DataFrame(trainy)
#n_values = train_labels.idxmax(axis=1)
y_values=np.argmax(train_labels,axis=1)
#print(y_values)
test_labels=np.array(testy)
yt_values=np.argmax(test_labels,axis=1)
#train_predictions_baseline = model.predict_classes(train_features, batch_size=150)
a=model.predict([test_features])
predict_classes=np.argmax(a,axis=1)
th=.8
atempo=a.max(axis=1)
print(atempo.shape)
print('Rate of Probability of classification exceeding a threshold in test',(atempo[atempo>th].shape[0])/atempo.shape[0])
#prob2=pd.DataFrame(model.predict_proba(test_features,batch_size=150))
#a=prob2.max(axis=1)
#print('Probability of classification',(a[a>.80].shape[0])/prob2.shape[0])
#print('Rate of Probability of classification exceding a trhershold',(a[a>.50].shape[0])/a.shape[0])
a1=model.predict([train_features])
predict_classes1=np.argmax(a1,axis=1)
#f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
#test_predictions_baseline = model.predict_classes(test_features, batch_size=150)
#print(test_predictions_baseline)
#f1_test=sklearn.metrics.f1_score(test_labels, test_predictions_baseline, average="weighted")
#print('f1_scores in testing set',f1_test)
#Calculating metrics for each class
print("EVALUATION ON TESTING DATA")
print(classification_report(yt_values, predict_classes))

C:\Users\User\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_517
Received: inputs=('Tensor(shape=(32, 400, 10))',)



2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step 

C:\Users\User\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_517
Received: inputs=('Tensor(shape=(None, 400, 10))',)



3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step
(74,)
Rate of Probability of classification exceeding a threshold in test 0.8648648648648649
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.35      0.40      0.38        15
           1       0.39      0.30      0.34        30
           2       0.42      0.53      0.47        15
           3       0.40      0.43      0.41        14

    accuracy                           0.39        74
   macro avg       0.39      0.42      0.40        74
weighted avg       0.39      0.39      0.39        74



In [436]:
cm=confusion_matrix(yt_values, predict_classes)
print(cm)
good=np.diag(cm)/np.unique(yt_values,return_counts=True)[1]
print(good)

[[6 4 3 2]
 [7 9 8 6]
 [1 5 8 1]
 [3 5 0 6]]
[0.4        0.3        0.53333333 0.42857143]


In [437]:
import plotly.figure_factory as ff
# invert z idx values

z= cm

# invert z idx values
#z = z[::-1]

x= [0,1,2,3]
y1=x
#y =x[::-1].copy() # invert idx values of x
#print(y)
#y=x
# change each element of z to type string for annotations
z_text = [[str(y1) for y1 in x] for x in z]

# set up figure
fig = ff.create_annotated_heatmap(z, x=x, y=y1, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Actual value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()